In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [2]:
import tensorflow as tf

2025-11-29 19:24:39.553033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-29 19:24:39.577497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-29 19:24:39.585385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-29 19:24:39.607579: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 19:24:40.622894: W tensorflow/compiler/tf2

In [3]:
from mobilenetv2ssd.core.config import load_config

In [4]:
from typing import Any

In [5]:
from mobilenetv2ssd.models.ssd.ops.loss_ops_tf import multibox_loss

## Config Files To Test Orchestration

In [6]:
main_cfg_path = "configs/train/default.yaml"
model_cfg_path = "configs/model/mobilenetv2_ssd_voc.yaml"
data_cfg_path = "configs/data/voc_224.yaml"
eval_cfg_path = "configs/eval/default.yaml"

In [7]:
config = load_config(main_cfg_path,model_cfg_path,data_cfg_path,eval_cfg_path)

## Helper Functions For Orchestration

In [9]:
def _extract_information_from_train_config(config : dict[str, Any]):
    train_config = config['train']
    model_config = config['model']
    loss_config = train_config.get("loss",{})
    loss_config = {
        "cls_loss_type": loss_config.get("cls_loss_type","ce_softmax"),
        "reg_loss_type": loss_config.get("reg_loss_type","smooth_l1"),
        "smooth_l1_beta": loss_config.get("smooth_l1_beta",1.0),
        "bbox_norm": loss_config.get("bbox_norm","none"),
        "from_logits": loss_config.get("from_logits",False),
        "ignore_index": loss_config.get("ignore_index",-1),
        "use_sigmoid": loss_config.get("use_sigmoid",False),
        "classification_weights": loss_config.get("cls_weight",1.0),
        "localization_weights": loss_config.get("reg_weight",1.0),
        "normalization_denom": loss_config.get("normalization",{}).get("type","num_pos"),
        "num_classes": model_config.get("num_classes",1),
        "reduction": loss_config.get("reduction","sum")
    }

    return loss_config

In [10]:
_extract_information_from_train_config(config)

{'cls_loss_type': 'softmax_ce',
 'reg_loss_type': 'smooth_l1',
 'smooth_l1_beta': 1.0,
 'bbox_norm': 'none',
 'from_logits': True,
 'ignore_index': -1,
 'use_sigmoid': False,
 'classification_weights': 1.0,
 'localization_weights': 1.0,
 'normalization_denom': 'num_pos',
 'num_classes': 21,
 'reduction': 'sum'}

## Orchestration Function

In [21]:
def calculate_final_loss(config: dict[str,Any], predicted_offsets: tf.Tensor, predicted_logits: tf.Tensor, localization_targets: tf.Tensor, classification_targets: tf.Tensor, positive_mask: tf.Tensor, negative_mask: tf.Tensor):
    # This is the function that calculates the multibox loss for the model
    # Steps:
    # 1. Get values from the config.
    # 2. Calculate the multibox loss

    loss_config = _extract_information_from_train_config(config)
    # loss_dict = tf.map_fn(lambda inputs: multibox_loss(predicted_offsets = inputs[0], predicted_logits = inputs[1], target_offsets = inputs[2], target_labels = inputs[3], positive_mask = inputs[4], negative_mask = inputs[5], localization_weight = loss_config["localization_weights"], classification_weight = loss_config["classification_weights"], beta = loss_config["smooth_l1_beta"], cls_loss_type = loss_config["cls_loss_type"],loc_loss_type = loss_config["reg_loss_type"], normalize_denom = loss_config["normalization_denom"], reduction = loss_config['reduction']),
    #                       elems = (predicted_offsets,predicted_logits,localization_targets,classification_targets,positive_mask,negative_mask),
    #                       fn_output_signature = {
    #                           "total_loss": tf.TensorSpec(shape=(), dtype=tf.float32),
    #                           "loc_loss": tf.TensorSpec(shape=(), dtype=tf.float32),
    #                           "cls_loss": tf.TensorSpec(shape=(), dtype=tf.float32),
    #                           "num_pos": tf.TensorSpec(shape=(), dtype=tf.int32),
    #                           "num_negative": tf.TensorSpec(shape=(), dtype=tf.int32)
    #                       }
    #                      )

    loss_dict = multibox_loss(predicted_offsets = predicted_offsets, predicted_logits = predicted_logits, target_offsets = localization_targets, target_labels = classification_targets, positive_mask = positive_mask, negative_mask = negative_mask, localization_weight = loss_config["localization_weights"], classification_weight = loss_config["classification_weights"], beta = loss_config["smooth_l1_beta"], cls_loss_type = loss_config["cls_loss_type"],loc_loss_type = loss_config["reg_loss_type"], normalize_denom = loss_config["normalization_denom"], reduction = loss_config['reduction'])

    return loss_dict

In [22]:
predicted_offsets = tf.constant([
    # Image 0 priors 0..4
    [
        [0.10,  0.05,  0.02, -0.01],  # prior 0
        [0.00, -0.03,  0.01,  0.04],  # prior 1
        [0.20, -0.10,  0.05,  0.02],  # prior 2
        [-0.05, 0.02,  0.01,  0.00],  # prior 3
        [0.03,  0.00, -0.02,  0.01],  # prior 4
    ],
    # Image 1 priors 0..4
    [
        [0.02,  0.01,  0.00, -0.01],  # prior 0
        [-0.01, 0.00,  0.03,  0.02],  # prior 1
        [0.15, -0.05,  0.04,  0.01],  # prior 2
        [0.00, -0.02,  0.00,  0.03],  # prior 3
        [-0.03, 0.01, -0.01,  0.00],  # prior 4
    ],
], dtype=tf.float32)

target_offsets = tf.constant([
    # Image 0
    [
        [0.12,  0.04,  0.01, -0.02],  # prior 0 (positive)
        [0.00,  0.00,  0.00,  0.00],  # prior 1 (neg)
        [0.18, -0.08,  0.06,  0.01],  # prior 2 (positive)
        [0.00,  0.00,  0.00,  0.00],  # prior 3 (neg)
        [0.00,  0.00,  0.00,  0.00],  # prior 4 (neg)
    ],
    # Image 1
    [
        [0.00,  0.00,  0.00,  0.00],  # prior 0 (neg)
        [0.00,  0.00,  0.00,  0.00],  # prior 1 (neg)
        [0.14, -0.06,  0.05,  0.02],  # prior 2 (positive)
        [0.00,  0.00,  0.00,  0.00],  # prior 3 (neg)
        [0.00,  0.00,  0.00,  0.00],  # prior 4 (neg)
    ],
], dtype=tf.float32)

predicted_logits = tf.constant([
    # Image 0
    [
        [ -1.0,  0.5,  1.2, -0.3],  # prior 0
        [  2.0, -1.0, -0.5, -0.2],  # prior 1
        [ -2.0,  1.5,  0.8,  0.0],  # prior 2
        [  1.0, -0.5, -0.2, -0.1],  # prior 3
        [  1.5, -0.8, -0.3, -0.4],  # prior 4
    ],
    # Image 1
    [
        [  2.2, -0.7, -0.4, -0.3],  # prior 0
        [  1.8, -0.6, -0.3, -0.5],  # prior 1
        [ -1.5,  1.0,  0.9, -0.2],  # prior 2
        [  2.0, -1.0, -0.5, -0.3],  # prior 3
        [  1.7, -0.9, -0.4, -0.6],  # prior 4
    ],
], dtype=tf.float32)

target_labels = tf.constant([
    # Image 0 priors 0..4
    [2, 0, 1, 0, 0],
    # Image 1 priors 0..4
    [0, 0, 3, 0, 0],
], dtype=tf.int32)

positive_mask = tf.constant([
    # Image 0: priors 0 and 2 are positive
    [ True, False,  True, False, False],
    # Image 1: prior 2 is positive
    [False, False,  True, False, False],
], dtype=tf.bool)

negative_mask = tf.constant([
    # Image 0: priors 1, 3, 4 selected as negatives
    [False,  True, False,  True,  True],
    # Image 1: priors 0, 1, 3, 4 selected as negatives
    [ True,  True, False,  True,  True],
], dtype=tf.bool)

In [23]:
calculate_final_loss(config, predicted_offsets = predicted_offsets , predicted_logits = predicted_logits , localization_targets = target_offsets, classification_targets = target_labels, positive_mask = positive_mask, negative_mask = negative_mask) 

{'total_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.7698979>,
 'loc_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.00035>,
 'cls_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.7695479>,
 'num_pos': <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 'num_negative': <tf.Tensor: shape=(), dtype=int32, numpy=7>}